In [43]:

# Import all modules and define frequently used functions

import os, csv
import math
from rdkit import Chem
from rdkit.Chem import Descriptors as CD

def get_text_from_file(file_name):

   f = open(file_name, "r")
   lines = f.readlines()
   f.close()

   return lines




In [45]:
# CONSTANTS: Define X as the "main atom" of interest in the functional group (B and Br are implemented), if a file prefix was used define the variable correspondingly, and finally define the folder path.

PATH = "/Users/lhoff/Desktop/calculations/results/inp_220201/"
X = "B"
file_prefix = "BA"
output_name = "output_file_name"



In [46]:
# This block first reads in th smiles from the dataset file used for the featurization, then it uses RDkit to get molecular weights and a custom parser to extract 
# the computed featurization data from the orca output files. Then it goes through the results.csv file which should be present in the results folder and contains
# additional data, e.g. Vbur, etc.

os.chdir(PATH)   # <<------- PATH points at the root folder of all results

collected_data = {}
smiles = get_text_from_file("dataset.txt")

for i in range(len(smiles)):
    smile = smiles[i].rstrip("\n")
    mol = Chem.MolFromSmiles(smile)
    mol_weight = CD.ExactMolWt(mol)
    collected_data[int(i)] = {"SMILES": smile}
    collected_data[int(i)]["MolWeight"] = float(mol_weight)




params_of_interest = [
    {
        "keyword": "FINAL SINGLE POINT ENERGY",
        "abbr": "FSPE",
        "y_offset": 0,
        "keep_index": [4]
    }, 
    {
        "keyword": "DIPOLE MOMENT",
        "abbr": "DIPOLE",
        "y_offset": 9,
        "keep_index": [3]
    }, 
    {
        "keyword": "CHEMICAL SHIELDING SUMMARY",
        "abbr": "CHEM SHIELD",
        "y_offset": 6,
        "keep_index": [0, 1, 2]
    }, 
    {
        "keyword": "HIRSHFELD ANALYSIS",
        "abbr": "HIRSHFELD",
        "y_offset": 7,
        "keep_index": [0, 1, 2]
    },
    {
        "keyword": "LOEWDIN ATOMIC CHARGES",
        "abbr": "LOEWDIN",
        "y_offset": 2,
        "keep_index": [0, 1, 3]
    }, 
    {
        "keyword": "MULLIKEN ATOMIC CHARGES",
        "abbr": "MULLIKEN",
        "y_offset": 2,
        "keep_index": [0, 1, 3]
    }, 
    {
        "keyword": "MAYER POPULATION ANALYSIS",
        "abbr": "MAYER",
        "y_offset": 11,
        "keep_index": [0, 1, 3]
    }, 
    {
        "keyword": "ORBITAL ENERGIES",
        "abbr": "HOMO/LUMO",
        "y_offset": 4,
        "keep_index": [0, 1] # not really keeping two, but using this as exception
    }, 
]

os.chdir("results")

for key in collected_data.keys():
    
    os.chdir(str(key) + "/orca")
    results = get_text_from_file("out_param_" + str(key) + ".out")

    number_of_atoms = int(get_text_from_file(file_prefix + "_" + str(key) + ".xyz")[0].split()[0])

    for i in range(len(results)): # for every line of text in the orca.out file

        for param in params_of_interest: # we check against every defined parameter of interest
            if param["keyword"] in results[i]: # if the defined keyword is contained in the line of text
                
                if len(param["keep_index"]) == 1:
                    arr = results[i + param["y_offset"]].split()
                    collected_data[key][param["abbr"]] = float(arr[param["keep_index"][0]])
                    
                elif len(param["keep_index"]) == 2:
                    
                    for j in range(9999):
                        arr = results[i + param["y_offset"] + j].split()
                            
                        if float(arr[1]) == 0:
                            
                            previous_arr = results[i + param["y_offset"] + j - 1].split()
                            
                            homo = float(previous_arr[3])
                            lumo = float(arr[3])

                            collected_data[key]["HOMO"] = homo
                            collected_data[key]["LUMO"] = lumo

                            collected_data[key]["E NEG"] = -0.5 * (lumo + homo)
                            collected_data[key]["HARDNESS"] = 0.5 * (lumo - homo)
                            collected_data[key]["SOFTNESS"] = 1.0/collected_data[key]["HARDNESS"]

                            break

                elif len(param["keep_index"]) == 3:

                    collected_data[key][param["abbr"]] = {}

                    for j in range(number_of_atoms):
                        arr = results[i + param["y_offset"] + j].split()

                        if len(arr) == 0: # fix for NMR spectra potentially not having data for all atoms
                            break

                        elif len(arr) == 3: # fix for LOEWDIN charges with 2 letter atoms
                            arr.append(arr[len(arr) - 1])
                            arr[1] = arr[1][:2]

                        atom_index = arr[param["keep_index"][0]]
                        
                        collected_data[key][param["abbr"]][int(atom_index)] = {

                            "atom_type": arr[param["keep_index"][1]],
                            "value": float(arr[param["keep_index"][2]])

                        }

                else:
                    print("weird" + str(arr))
                
                                
    os.chdir("../../")


t = open("results.csv", "r")
tt = t.readlines()
t.close()


properties = tt[0].split(",")
properties.pop(0)
properties[len(properties)-1] = properties[len(properties)-1][0:len(properties[len(properties)-1])-1]

tt.pop(0)

for line in tt:
    values = line.split(",")
    values[len(values)-1] = values[len(values)-1][0:len(values[len(values)-1])-1]

    key = int(values[0])
    values.pop(0)

    for i, value in enumerate(values):
        collected_data[key][properties[i]] = float(value)

In [47]:
#This block finds the relevant carbon atoms by finding the carbon atom closest to the atom of interest defined at the top of this notebook. Based on the closest carbon atom indeces
# it compiles a plot_data dictionary which only holds relevant data.

closest_carbons = {}

for key in collected_data.keys():
    
    os.chdir(str(key) + "/orca")
    coordinates = get_text_from_file(file_prefix + "_" + str(key) + ".xyz")

    #number_of_atoms = int(coordinates[0].split()[0])

    coordinates.pop(0)
    coordinates.pop(0)

    atom_coord = [] # will hold the Vec3D coordinates of the boron
    min_dist = 9999999999999 # will hold the coordinates of the carbon closest to the boron
    carbon_index = -1

    for i in range(len(coordinates)):

        arr = coordinates[i].split()
        if arr[0] == X:
            arr.pop(0)
            atom_coord = [float(arr[0]), float(arr[1]), float(arr[2])]
            break

    for i in range(len(coordinates)):
        arr = coordinates[i].split()

        if arr[0] == "C":
        
            vec3d = [atom_coord[0] - float(arr[1]), atom_coord[1] - float(arr[2]), atom_coord[2] - float(arr[3])]

            length = math.sqrt((vec3d[0] * vec3d[0]) + (vec3d[1] * vec3d[1]) + (vec3d[2] * vec3d[2]))

            if length < min_dist:
                min_dist = length
                carbon_index = i
    
    closest_carbons[key] = {
        
        "dis": min_dist,
        "index": carbon_index

    }

    os.chdir("../../")



plot_data = {}

incomplete_entries = []

for key in collected_data.keys():
    plot_data[key] = {}
    index = closest_carbons[key]["index"]
    for prop in collected_data[0].keys():
        if prop in collected_data[key].keys():
            if type(collected_data[key][prop]) is not dict:
                plot_data[key][prop] = collected_data[key][prop]
            elif type(collected_data[key][prop]) is dict:
                plot_data[key][prop] = collected_data[key][prop][index]["value"]
        else:
            print(f"BA{key} is missing the {prop} property")
            if key not in incomplete_entries:
                incomplete_entries.append(key)

    plot_data[key]["CB DIST"] = closest_carbons[key]["dis"]

for entry in incomplete_entries:
    print(f"Entry {entry} was removed from plot data due to incomplete data")
    plot_data.pop(entry)
    smiles.pop(entry)

print(f"{len(incomplete_entries)} entries were removed due to incomplete data")

BA511 is missing the Vbur property
BA604 is missing the Vbur property
BA967 is missing the HOMO property
BA967 is missing the LUMO property
BA967 is missing the E NEG property
BA967 is missing the HARDNESS property
BA967 is missing the SOFTNESS property
BA967 is missing the MULLIKEN property
BA967 is missing the LOEWDIN property
BA967 is missing the MAYER property
BA967 is missing the HIRSHFELD property
BA967 is missing the FSPE property
BA967 is missing the DIPOLE property
BA967 is missing the CHEM SHIELD property
BA1424 is missing the Vbur property
BA1565 is missing the Vbur property
BA1872 is missing the Vbur property
BA1960 is missing the Vbur property
BA2321 is missing the Vbur property
BA2458 is missing the Vbur property
BA2464 is missing the Vbur property
BA2684 is missing the Vbur property
Entry 511 was removed from plot data due to incomplete data
Entry 604 was removed from plot data due to incomplete data
Entry 967 was removed from plot data due to incomplete data
Entry 1424 

In [40]:
# Finally, the plot data dictionary is converted in to a .csv file. Make sure to define a name for the file.

column_titles = []

for key in plot_data[0].keys():
    column_titles.append(key)

file = f"{output_name}.csv"  # <------ DEFINE A NAME FOR THE CSV DATA FILE

with open(file, "w") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=column_titles)
    writer.writeheader()
    for key in plot_data.keys():
        writer.writerow(plot_data[key])